# vLLM 서비스 endpoint에 inference 진행

In [ ]:
import json
import requests

my_outer_ip = "104.199.178.22"

def apply_body(prompt, system_prompt="You are an AI Assistant called Gauss."):
    body = {
        "model": "models/Qwen2.5-1.5B-Instruct/",
        "messages": [
            {
                "content": system_prompt, 
                "role": "system"
            }, 
            {
                "content": prompt,
                "role": "user"
            }
        ],
        "max_tokens": 512,
        "seed": 12345, 
    }
    return body

### vLLM 서버 정상작동 확인

In [ ]:
response = requests.get(f"http://{my_outer_ip}:8006/health")
response

### vLLM 서버에 추론요청 보내기

In [19]:
prompt = "Hello, what is your name?"

system_prompt = "You are an AI Assistant called Gauss."

body = apply_body(prompt, system_prompt)
headers = {"Content-Type": "application/json"}
endpoint = f"http://{my_outer_ip}:8006/v1/chat/completions"


response = requests.post(endpoint, data=json.dumps(body), headers=headers)

### 모델이 받은 원문 확인

In [15]:
from transformers import AutoTokenizer
model_path = "./models/Qwen2.5-1.5B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(model_path)
print(tokenizer.apply_chat_template(body['messages'], tokenize=False))

<|im_start|>system
You are an AI Assistant called Gauss.<|im_end|>
<|im_start|>user
Hello, what is your name?<|im_end|>



In [20]:
return_body = json.loads(response.content.decode())
print(return_body["choices"][0]['message']['content'])

Hello! I am called Gauss. It's nice to meet you. I am an AI model created to perform mathematical calculations and complex algorithms. Am I helping you with anything?


# 부하 테스트 툴 locust 사용한 성능 테스트

In [ ]:
!pip install locust

아래 명령어 실행:
> locust -f locust_default.py --host http://localhost:8006 \
> locust -f locust_add_check.py --host http://localhost:8006